In [ ]:
pip install underthesea


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import string
from underthesea import word_tokenize

In [ ]:
# -----------------------------
# CONFIG: Bộ keywords lý do churn
# -----------------------------

CHURN_KEYWORDS = {
    'bug': ['bug', 'lỗi', 'crash', 'sập', 'đơ', 'treo'],
    'lag': ['lag', 'giật', 'chậm', 'disconnect', 'delay', 'ping cao', 'mất kết nối', 'server lỗi'],
    'ads': ['quảng cáo', 'ads'],
    'update': [
        'update', 'cập nhật', 'bắt update', 'tải lại', 'dung lượng',
        'dung lượng lớn', 'nặng máy', 'nặng', 'full gb', 'quá nặng', 'chiếm bộ nhớ'
    ],
    'price': ['đắt', 'mua', 'nạp', 'tiền', 'nạp thẻ'],
    'toxic': [
        'toxic', 'trẩu', 'phá game', 'afk', 'feed', 'chửi', 'team ngu',
        'trẻ trâu', 'phá trận', 'tố cáo', 'bus bẩn', 'gạ', 'quấy rối'
    ],
    'hack': [
        'hack', 'cheat', 'tool', 'bug map', 'mod', 'hack map',
        'buff elo', 'ddos', 'gian lận'
    ],
    'matchmaking': [
        'ghép trận', 'matchmaking', 'ghép team', 'ghép rank', 'thuật toán',
        'team ngu', 'rank lỗi', 'kda sai', 'tính kda', 'trừ uy tín',
        'sét rank', 'reset rank', 'elo', 'ghép ngẫu nhiên'
    ],
    'performance': ['chậm', 'giật', 'lag', 'nặng', 'thiết bị yếu', 'yếu máy'],
    'forced_pick': ['chọn gì phải chơi đó', 'tính năng mới', 'bắt buộc', 'ép buộc', 'meta', 'bị ép pick'],
    'trash': ['rác', 'trash', 'tệ', 'cức', 'quá chán', 'quá tệ', 'vứt đi', 'vô dụng']
}

In [ ]:
# -----------------------------
# UTILS: Các hàm xử lý
# -----------------------------

def clean_text(text: str) -> str:
    """
    Làm sạch review:
    - lowercase
    - bỏ số, dấu câu, khoảng trắng thừa
    """
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def tokenize_vi(text: str) -> str:
    """
    Tokenize tiếng Việt bằng underthesea
    """
    return word_tokenize(text, format="text")


def detect_sentiment(score: int) -> str:
    """
    Sentiment rule:
    - score < 3: Negative
    - score = 3: Neutral
    - score > 3: Positive
    """
    if score < 3:
        return 'Negative'
    elif score == 3:
        return 'Neutral'
    return 'Positive'


def detect_churn_reason(text: str, score: int, keywords_dict: dict) -> str:
    """
    Detect churn_reason:
    - Nếu match keywords: trả về nhóm match
    - Nếu không match nhưng score < 3: trả về 'other'
    - Nếu không match & không tiêu cực: trả về 'none'
    """
    reasons = []
    for reason, keywords in keywords_dict.items():
        for kw in keywords:
            if kw in text:
                reasons.append(reason)
                break

    if reasons:
        return ', '.join(sorted(set(reasons)))
    if score < 3:
        return 'other'
    return 'none'


In [ ]:
# -----------------------------
# MAIN PIPELINE
# -----------------------------

def enrich_reviews(input_file: str, output_file: str) -> None:
    """
    Pipeline đầy đủ:
    - Load CSV
    - Clean, tokenize
    - Gán sentiment
    - Detect churn_reason
    - Xuất file enrich
    - In sample mỗi bước
    """
    df = pd.read_csv(input_file)
    print(f"✅ Loaded: {df.shape[0]} rows | {df.shape[1]} columns")
    print(df.head(3)[['user_name', 'review_text', 'score']])

    # 1. Chuyển review_date
    df['review_date'] = pd.to_datetime(df['review_date'])
    print("\n✅ Converted `review_date`:")
    print(df[['review_date']].head(3))

    # 2. Clean text
    df['clean_review'] = df['review_text'].astype(str).apply(clean_text)
    print("\n✅ Cleaned `review_text`:")
    print(df[['review_text', 'clean_review']].head(3))

    # 3. Tokenize
    df['tokens'] = df['clean_review'].apply(tokenize_vi)
    print("\n✅ Tokenized text:")
    print(df[['clean_review', 'tokens']].head(3))

    # 4. Sentiment
    df['sentiment'] = df['score'].apply(detect_sentiment)
    print("\n✅ Sentiment:")
    print(df[['score', 'sentiment']].head(3))

    # 5. Churn Reason
    df['churn_reason'] = df.apply(
        lambda row: detect_churn_reason(row['clean_review'], row['score'], CHURN_KEYWORDS),
        axis=1
    )
    print("\n✅ Churn Reason:")
    print(df[['clean_review', 'churn_reason']].head(5))

    # 6. Save
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n✅ Pipeline DONE! File enrich đã lưu: {output_file}")


if __name__ == "__main__":
    enrich_reviews(
        input_file="lien_quan_mobile_filtered_latest_versions.csv",
        output_file="lien_quan_mobile_churn_enriched.csv"
    )

✅ Loaded: 7470 rows | 8 columns
       user_name                                     review_text  score
0  Thái Thành Jr  Game cập nhật như cức, lại phải tải lại tất cả      1
1      Giang Bùi                                    Game hay 😊😊😊      5
2       Chau Bui                                  ko nói nên lời      1

✅ Converted `review_date`:
          review_date
0 2025-07-10 14:27:01
1 2025-07-10 14:26:30
2 2025-07-10 14:25:52

✅ Cleaned `review_text`:
                                      review_text  \
0  Game cập nhật như cức, lại phải tải lại tất cả   
1                                    Game hay 😊😊😊   
2                                  ko nói nên lời   

                                    clean_review  
0  game cập nhật như cức lại phải tải lại tất cả  
1                                   game hay 😊😊😊  
2                                 ko nói nên lời  

✅ Tokenized text:
                                    clean_review  \
0  game cập nhật như cức lại phải tải lại tất cả  